# Parameter homotopies

Consider the situation in which one has to solve a specific instance of a *parametrized* family of polynomial systems


$$
P = \{F(x,p)  \mid p \in \mathbb{C}^m\}.
$$

It is a rule of thumb that for solving systems with parameters one should **first solve the system for a random set of complex parameters $q$**. Only after this one should track the solutions towards the parameters $p$ one is interested in by using the *parameter homotopy*

$$H(x,t) := F(x, (1-t)p + tq)$$

For instance, let $P = \{F(x,y,a,b) \mid (a,b)\in\mathbb{C}^2\}$ where

$$F(x,y,a,b) = \begin{bmatrix} x^2-a \\ xy-a+b \end{bmatrix}.$$

Suppose we are interested in solving the system given by $(a,b)=(3,5)$. Then, following the rule above we solve it as follows.

In [ ]:
using HomotopyContinuation
@polyvar x y a b
F = [x^2 - a, x * y - a + b]

random_complex_parameters = randn(ComplexF64,2)
S0 = solve([subs(f, [a,b] => random_complex_parameters) for f in F])

Then, we track the solutions `S0` towards $(a,b)=(3,5)$ using a parameter homotopy.

In [ ]:
S = solve(F, solutions(S0), 
        parameters = [a,b], 
        start_parameters = random_complex_parameters, 
        target_parameters = [3,5])

## Tracking towards many parameters

The above computational scheme is particularly useful when one has to solve a system for many different sets of parameters.

Here is an example: consider the real algebraic variety $V=\{f=0\}\subset\mathbb{R}^2$, where

$$f(x,y) = x^2y + 2xy - y^3,$$

and consider the problem of generating points on $V$ (for instance as input for persistence homology). 

A way of getting points on $V$ is to intersect it with varying *real* linear spaces and record the real intersection points. In the above framework, the coefficients of the linear space would be the parameters.

Let us set up the system in Julia and solve it for a random *complex* linear space.

In [ ]:
using HomotopyContinuation
@polyvar x y
@polyvar a b c # variables for the linear space
p = [a, b, c] # collect the parameters in a vector

F = [x^2 * y + 2x * y - y^3 + 1; a * x + b * y + c]

# now we solve one particular instance for a,b,c complex. we use this as start system
p₀ = randn(ComplexF64, 3)
F₀ = [subs(f, p => p₀) for f in F]
complex_result = solve(F₀)
S_p₀ = solutions(complex_result)

As before, we can now track the complex parameters towards parameters we are interested in. Here, HomotopyContinuation.jl has a distinguished feature: we can create an object `tracker` and manipulate it for tracking efficiently many parameters.

In [ ]:
tracker = pathtracker(F; parameters=p, start_parameters=p₀, target_parameters = randn(3))

# we compute 1000 random intersections
data = [randn(3) for _ in 1:1000]
points = Vector{Vector{Float64}}()
for pp in data
    for s in S_p₀
        result = track(tracker, s; target_parameters=pp)
        # check that the tracking was successfull and that we have a real solution
        if is_success(result) && is_real(result)
            # only store the solutions
            s = real(solution(result))
            push!(points, s)
        end
    end
end

Here is the result:

In [ ]:
using Plots
points_as_matrix = hcat(points...)
scatter(points_as_matrix[1,:], points_as_matrix[2,:], 
        xlims = (-10,10), ylims = (-10,10), 
        markersize = 2,
        markercolor = :steelblue,
        markerstrokealpha = 0.0)

## Parameter homotopies of composite systems

You can also track a parameter homotopies for composite system. If

$$ f = \begin{bmatrix} ab - q\\  ac - p\end{bmatrix}, \quad g =  \begin{bmatrix}x + y\\ y + 3\\ x + 2\end{bmatrix},$$

and you want to track solutions from random complex parameters to $(p,q) = (2, 3)$, this is how it works:

In [ ]:
@polyvar a b c x y 
@polyvar p q
f = [a * b - q, a * c - p]
g = [x + y, y + 3, x + 2]

complex_parameters = randn(ComplexF64, 2)
f_complex = [subs(fi, [p, q] => complex_parameters) for fi in f]
S_complex = solve(f_complex ∘ g)

Then, we track the solution in `S_complex` to the parameters $(2,3)$.

In [ ]:
S = solve(f ∘ g, solutions(S_complex), 
        parameters=[p, q], 
        start_parameters = complex_parameters, 
        target_parameters=[2, 3])

It does not matter at which level the parameters are.